
# Hubble law

In [ ]:

using CSV
using DataFrames
using PyPlot

In [ ]:

# url = "https://vizier.u-strasbg.fr/viz-bin/asu-txt?-source=J/ApJ/716/712/tableb2&-out=SN&-out=zCMB&-out=mu"
url = "https://cdsarc.u-strasbg.fr/viz-bin/asu-txt?-source=J/ApJ/716/712/tableb2&-out=SN&-out=zCMB&-out=mu"

In [ ]:

catalog = download(url)

In [ ]:

df = CSV.read(catalog, DataFrame, skipto=38, delim=' ', ignorerepeated=true,
              types=[String, Float64, Float64], silencewarnings=true,
              header=["name", "redshift", "modulus"],)

In [ ]:

dropmissing!(df)

In [ ]:

filter!(row -> row.redshift < 0.04, df)

In [ ]:

dist(modulus) = 10.0 ^ (modulus / 5 - 5)

In [ ]:

sort!(df, "modulus");

In [ ]:

df

In [ ]:

distances = dist.(df.modulus);

In [ ]:

plot(distances, df.redshift, ".", label="measurements")
grid(true)
legend()
ylabel("Red Shift")
xlabel("Distance (Mpc)");

In [ ]:

"""
   alpha, beta, sigma = linear_regression(x, y)

Least square linear regression fit y = alpha + beta * x
Sigma is standard deviation for beta
"""
function linear_regression(x, y)
    n = length(x)
    xbar = sum(x)/n
    ybar = sum(y)/n
    denom = sum((x .- xbar).^2)
    beta = sum((x .- xbar).*(y .- ybar))/denom
    alpha = ybar - beta*xbar
    sigma = sqrt(sum((y .- alpha .- beta*x).^2)/((n - 2)*denom))
    return alpha, beta, sigma
end

In [ ]:

alpha, beta, sigma = linear_regression(distances, df.redshift)

In [ ]:

plot(distances, df.redshift, ".", label="measurements")
plot(distances, alpha .+ beta*distances, label="LSq linear fit")
grid(true)
legend()
ylabel("Red Shift")
xlabel("Distance (Mpc)");

In [ ]:

const c = 300000.0  # speed of light, km/sec
H0 = c * beta
round(H0, digits=3)

In [ ]:

const mpc = 3.09e19     # 1 megaparsec in km
h0 = H0 / mpc
round(h0, sigdigits=3)

In [ ]:

Th = 1 / h0
round(Th, sigdigits=3)

In [ ]:

round(Th/(60*60*24*365), sigdigits=2)